In [2]:
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import json
import pickle
import http.client
import certifi
import ssl
import os
import altair as alt
import pytz
import numpy as np
tz = pytz.timezone('America/Edmonton')

In [ ]:
with open('./current_df.pickle', 'rb') as token:
    current_df = pickle.load(token)
current_df = current_df[1]
current_df

In [12]:
#@st.experimental_singleton()
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate(st.secrets["gcp_service_account"])
app = firebase_admin.initialize_app(cred)
db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [400]:
# Set path to Firestore DB
currentData_ref = db.collection(u'appData').document('currentData')
# Update current_df data in Firestore DB
currentData_ref.set(current_df.to_dict('list'))

dict_keys(['timeStamp', 'value', 'fuelType'])

In [11]:
# Set path to Firestore DB
currentData_ref = db.collection(u'appData').document('currentData')
# Read current_df from Firestore
current_df = pd.DataFrame.from_dict(currentData_ref.get().to_dict())
current_df['timeStamp'] = current_df['timeStamp'].dt.tz_convert('America/Edmonton') 
current_df

,value,timeStamp,fuelType
0,814.00,2022-10-12 00:00:00-06:00,Coal
1,818.00,2022-10-12 00:05:00-06:00,Coal
2,821.00,2022-10-12 00:10:00-06:00,Coal
3,817.00,2022-10-12 00:15:00-06:00,Coal
4,818.00,2022-10-12 00:20:00-06:00,Coal
...,...,...,...
5107,0.00,2022-10-13 11:05:00-06:00,Pool Price
5108,58.22,2022-10-13 11:10:00-06:00,Pool Price
5109,60.40,2022-10-13 11:15:00-06:00,Pool Price
5110,60.40,2022-10-13 11:20:00-06:00,Pool Price


In [57]:
with open('./monthly_df.pickle', 'rb') as token:
    monthly_df = pickle.load(token)
#daily_df = daily_df[0][1]
monthly_df


,timeStamp,value,fuelType
10,2022-11-01 00:00:00-06:00,50,Coal
11,2022-12-01 00:00:00-07:00,0,Coal
10,2022-11-01 00:00:00-06:00,1080,Natural Gas
11,2022-12-01 00:00:00-07:00,780,Natural Gas
10,2022-11-01 00:00:00-06:00,0,Dual Fuel
...,...,...,...
5,2024-06-01 00:00:00-06:00,30,Biomass & Other
6,2024-07-01 00:00:00-06:00,30,Biomass & Other
7,2024-08-01 00:00:00-06:00,30,Biomass & Other
8,2024-09-01 00:00:00-06:00,30,Biomass & Other


In [60]:
alert_dates = {'Biomass & Other':datetime.now(),'Coal':datetime.now(),'Dual Fuel':datetime.now(),'Energy Storage':datetime.now(),'Hydro':datetime.now(),
                'Natural Gas':datetime.now(),'Solar':datetime.now(),'Wind':datetime.now(),'BC':datetime.now(),'Saskatchewan':datetime.now(),'Montana':datetime.now(),
                'Intertie':datetime.now(),'Pool Price':datetime.now(),'3-Day Solar Forecast':datetime.now(),'7-Day Wind Forecast':datetime.now()}
with open('./alert_dates.pickle', 'wb') as handle:
    pickle.dump(alert_dates, handle, protocol=pickle.HIGHEST_PROTOCOL)
alert_dates

{'Biomass & Other': datetime.datetime(2022, 10, 13, 0, 39, 20, 683548),
 'Coal': datetime.datetime(2022, 10, 13, 0, 39, 20, 683558),
 'Dual Fuel': datetime.datetime(2022, 10, 13, 0, 39, 20, 683558),
 'Energy Storage': datetime.datetime(2022, 10, 13, 0, 39, 20, 683558),
 'Hydro': datetime.datetime(2022, 10, 13, 0, 39, 20, 683558),
 'Natural Gas': datetime.datetime(2022, 10, 13, 0, 39, 20, 683559),
 'Solar': datetime.datetime(2022, 10, 13, 0, 39, 20, 683559),
 'Wind': datetime.datetime(2022, 10, 13, 0, 39, 20, 683559),
 'BC': datetime.datetime(2022, 10, 13, 0, 39, 20, 683559),
 'Saskatchewan': datetime.datetime(2022, 10, 13, 0, 39, 20, 683559),
 'Montana': datetime.datetime(2022, 10, 13, 0, 39, 20, 683560),
 'Intertie': datetime.datetime(2022, 10, 13, 0, 39, 20, 683560),
 'Pool Price': datetime.datetime(2022, 10, 13, 0, 39, 20, 683560),
 '3-Day Solar Forecast': datetime.datetime(2022, 10, 13, 0, 39, 20, 683560),
 '7-Day Wind Forecast': datetime.datetime(2022, 10, 13, 0, 39, 20, 683560)}

In [14]:
def release_token(accessToken):
    path = '/api/ReleaseToken'
    server = 'api.nrgstream.com'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
with open('./accessToken.pickle', 'rb') as token:
    accessToken = pickle.load(token)
print(accessToken)
release_token(accessToken)

uWRqRKIwXjAWcPo5B4Wz6x_iSPWAu6prPdKWny7PKauiL7yqltm3CTE6vTmluQj1_pzVm5MkXBQSow57DatdjfMtNvJfZ9baGpy362xb9yq5XTLyidxbWv2_BRYqZrvdpnbp0sCd8xd172vCHyxae9IoPqhboEx8PBbzkY-_h155U9ckX3sjIRCOaXk0_ek8WMenLRF8mpMDH8PukPnOvQUWSwVd2wHi8qeB8JZlGC1uTkfPuexsXb4aFVj9_BYKS08wWiWIc6rQT46S1zocQcgYFq2bnYlB998QZdfhYVrBPC5SoWVh_WAoHIVwNsgLy1VEAUZVCX7bPmF73Xo_G2FrXHo


In [16]:
datetime.now(tz).date()-timedelta(days=7)

datetime.date(2022, 10, 6)